In [10]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from difflib import get_close_matches
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

In [13]:
!pip install textdistance

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import textdistance

In [5]:
from tqdm.notebook import tqdm

In [6]:
# Задание 1

In [7]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [8]:
len(true)

915

In [13]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [14]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [15]:
y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)
        
        y_pred.append(predict_mistaken(pair[1], vocab))

NameError: name 'align_words' is not defined

In [16]:
def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

In [17]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [18]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [19]:
def get_closest_hybrid_match_ranked(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein, vocab=vocab):
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    candidates_with_equal_distance = []

    for n in range(1, len(closest) + 1):
        equal_distance = len(set(dist for _, dist in closest[:n])) == 1
        if equal_distance:
            
            candidates_with_equal_distance.append(closest[n - 1])
        else:
            break

    if candidates_with_equal_distance: 
        
        candidates_with_equal_distance = [(word, dist, P(word)) for word, dist in candidates_with_equal_distance]
        
        candidates_with_equal_distance.sort(key=lambda x: x[2], reverse=True)

        remaining_candidates = closest[len(candidates_with_equal_distance):]

        merged_candidates = candidates_with_equal_distance + remaining_candidates

        return merged_candidates
    else:
        return closest  

In [20]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [21]:
def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [22]:
%%time
get_closest_hybrid_match('кстате', X, vec, topn=5, metric=textdistance.damerau_levenshtein)

CPU times: total: 109 ms
Wall time: 102 ms


[('статике', 0.5714285714285714),
 ('кастету', 0.5714285714285714),
 ('кастет', 0.5),
 ('касте', 0.5),
 ('секстета', 0.5)]

In [23]:
# 'статике' и 'кастету' - одинаковое расстояние 

In [24]:
get_closest_hybrid_match_ranked('кстате', X, vec, topn=5, metric=textdistance.damerau_levenshtein, vocab=vocab)

[('статике', 0.5714285714285714, 1.939759225446382e-07),
 ('кастету', 0.5714285714285714, 1.939759225446382e-07),
 ('кастет', 0.5),
 ('касте', 0.5),
 ('секстета', 0.5)]

In [25]:
# Задание 2

In [26]:
# словарь правильных слов
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [27]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [28]:
corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall(r'\w+', corpus.lower()))

print(len(vocab)) 

368802


In [29]:
# создание словаря удалений
def edits1(word):
    letters = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return deletes

delete_dict = {}
for word in vocab:
    deletions = edits1(word)
    for deletion in deletions:
        if deletion not in delete_dict:
            delete_dict[deletion] = word

print(len(delete_dict))

2872459


In [30]:
def correct_spelling_with_probability(word, delete_dict, vocab):
    word_deletes = edits1(word)

    possible_corrections = []
    
    for delete in word_deletes:
        
        if delete in delete_dict:
            possible_corrections.append(delete_dict[delete])
    
    if possible_corrections:
        corrected_word = max(possible_corrections, key=lambda x: P(x, N=N))
        return corrected_word
    
    return word

In [45]:
%%time
correct_spelling_with_probability('закомая', delete_dict, vocab)

CPU times: total: 0 ns
Wall time: 0 ns


'знакома'

In [47]:
%%time
correct_spelling_with_probability('жещин', delete_dict, vocab)

CPU times: total: 0 ns
Wall time: 0 ns


'лещин'

In [49]:
%%time
correct_spelling_with_probability('песпектива', delete_dict, vocab)

CPU times: total: 0 ns
Wall time: 0 ns


'перспектив'

In [50]:
%%time
correct_spelling_with_probability('измченная', delete_dict, vocab)

CPU times: total: 0 ns
Wall time: 0 ns


'низменная'

In [52]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/139.4 kB ? eta -:--:--
     --                                       10.2/139.4 kB ? eta -:--:--
     --------                              30.7/139.4 kB 330.3 kB/s eta 0:00:01
     --------                              30.7/139.4 kB 330.3 kB/s eta 0:00:01
     ------------------------------------ 139.4/139.4 kB 828.0 kB/s eta 0:00:00
                                              0.0/2.3 MB ? eta -:--:--
     -                                        0.1/2.3 MB 1.1 MB/s eta 0:00:03
     --                                       0.1/2.3 MB 1.2 MB/s eta 0:00:02
     ---                                      0.2/2.3 MB 1.5 MB/s eta 0:00:02
     ----                                     0.3/2.3 MB 1.4 MB/s eta 0:00:02
     -----                                    0.3/2.3 MB 1.5 MB/s eta 0:00:02
     ------                                   0.4/2.3 MB 1.5 MB/s eta 0:00

In [31]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = cashed.get(pair[1], correct_spelling_with_probability(pair[1], delete_dict, vocab))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]